<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Copia_de_Untitled10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simulador para punto de cambio bayesiano


Instrucciones
Entorno: Este código está diseñado para ejecutarse en un Jupyter Notebook o JupyterLab. Es la forma más sencilla de combinar el código de simulación (numpy), las gráficas (matplotlib) y los controles interactivos (ipywidgets).

Copiar y Pegar: Copia todo el bloque de código en una celda de tu notebook.

Ejecutar: Ejecuta la celda. Deberían aparecer los sliders de control y las gráficas de la simulación.

Interactuar: Mueve los sliders para cambiar las probabilidades de recompensa, la duración de los bloques o la tasa de peligro. Las gráficas se actualizarán automáticamente para mostrar cómo el modelo se adapta a las nuevas condiciones.

Cómo Interpretar los Resultados
Gráfico 1 (Superior): Verás la probabilidad real en rojo punteado. Los marcadores negros son las recompensas que el agente realmente observó (0 o 1). La línea verde es la creencia o predicción del agente. Observa cómo la línea verde intenta seguir a la roja, pero con un retraso.

Gráfico 2 (Medio): Esta es la tasa de aprendizaje (
alpha). Nota cómo es generalmente baja, pero se dispara justo después del punto de cambio (la línea vertical punteada). Esto permite que el modelo se adapte rápidamente a la nueva probabilidad de recompensa.

Gráfico 3 (Inferior): Esta es la creencia interna del modelo de que un cambio acaba de ocurrir (CPP). Observa la correlación perfecta: cuando el modelo se vuelve muy seguro de que ha ocurrido un cambio (pico en el gráfico 3), aumenta drásticamente su tasa de aprendizaje (pico en el gráfico 2).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, Layout
from IPython.display import display

# --- Lógica Central del Simulador ---

class ChangePointSimulator:
    """
    Simula un agente Bayesiano que aprende en un entorno con puntos de cambio.
    """
    def __init__(self, hazard_rate=0.01):
        if not (0 < hazard_rate < 1):
            raise ValueError("La Tasa de Peligro (Hazard Rate) debe estar entre 0 y 1.")

        # Parámetros del modelo
        self.H = hazard_rate  # Tasa de Peligro (probabilidad a priori de un cambio)
        self.V = 0.5          # Valor esperado inicial (creencia inicial)
        self.uncertainty = 1.0 # Incertidumbre inicial sobre el valor

        # Historial para graficar
        self.history = {
            'rewards': [],
            'predictions': [],
            'learning_rates': [],
            'cpp': [],
            'uncertainty': []
        }

    def _log_likelihood_bernoulli(self, outcome, prediction):
        """Calcula el logaritmo de la verosimilitud de un resultado Bernoulli."""
        pred = np.clip(prediction, 1e-6, 1 - 1e-6) # Evitar log(0)
        return outcome * np.log(pred) + (1 - outcome) * np.log(1 - pred)

    def update(self, reward):
        """
        Ejecuta un único ensayo de la simulación.
        """
        # 1. Almacenar la predicción *antes* de observar la recompensa
        self.history['predictions'].append(self.V)
        self.history['rewards'].append(reward)
        self.history['uncertainty'].append(self.uncertainty)

        # 2. Calcular la probabilidad del dato bajo la hipótesis de 'no cambio'
        log_likelihood_no_change = self._log_likelihood_bernoulli(reward, self.V)

        # 3. Calcular la probabilidad del dato bajo la hipótesis de 'cambio'
        #    Si hay un cambio, el modelo asume que no sabe nada (V=0.5)
        log_likelihood_change = self._log_likelihood_bernoulli(reward, 0.5)

        # 4. Calcular la Probabilidad de Punto de Cambio (CPP) usando la regla de Bayes
        #    Se usa el truco log-sum-exp para estabilidad numérica
        log_posterior_numerator = log_likelihood_change + np.log(self.H)
        log_posterior_denominator_part1 = log_likelihood_no_change + np.log(1 - self.H)

        # Log-Sum-Exp
        max_log = np.maximum(log_posterior_numerator, log_posterior_denominator_part1)
        log_denominator = max_log + np.log(np.exp(log_posterior_numerator - max_log) + np.exp(log_posterior_denominator_part1 - max_log))

        cpp = np.exp(log_posterior_numerator - log_denominator)
        self.history['cpp'].append(cpp)

        # 5. Calcular la Tasa de Aprendizaje (alpha) dinámica
        #    Alpha es una mezcla de la incertidumbre (si no hay cambio) y 1 (si hay cambio)
        alpha = cpp * 1.0 + (1 - cpp) * self.uncertainty
        self.history['learning_rates'].append(alpha)

        # 6. Calcular el Error de Predicción (PE) y actualizar el valor
        prediction_error = reward - self.V
        self.V = self.V + alpha * prediction_error

        # 7. Actualizar la incertidumbre
        #    La incertidumbre disminuye con cada aprendizaje, pero se resetea con el cambio.
        self.uncertainty = self.uncertainty * (1 - alpha) + 0.1 * cpp
        self.uncertainty = np.clip(self.uncertainty, 0.1, 1.0) # Acotar la incertidumbre


# --- Función para correr la simulación y graficar ---

def run_and_plot_simulation(prob_b1, len_b1, prob_b2, len_b2, hazard_rate):
    """
    Crea los bloques, corre la simulación y genera las gráficas.
    """
    # Definir la secuencia de probabilidades de recompensa
    reward_probabilities = np.concatenate([
        np.full(len_b1, prob_b1),
        np.full(len_b2, prob_b2)
    ])

    # Generar las recompensas estocásticas (0 o 1)
    rewards = np.random.binomial(1, reward_probabilities)

    # Inicializar y correr la simulación
    simulator = ChangePointSimulator(hazard_rate=hazard_rate)
    for reward in rewards:
        simulator.update(reward)

    # --- Graficar los resultados ---
    fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)
    fig.suptitle('Simulador de Modelo Bayesiano de Punto de Cambio', fontsize=16, y=0.95)
    trials = np.arange(1, len(rewards) + 1)

    # Gráfico 1: Recompensas y Predicciones del Modelo
    ax1 = axes[0]
    ax1.plot(trials, simulator.history['predictions'], 'g-', label='Predicción del Modelo ($V_t$)')
    ax1.scatter(trials, simulator.history['rewards'], c='k', marker='|', alpha=0.7, label='Recompensa Observada ($R_t$)')
    ax1.plot(trials, reward_probabilities, 'r--', alpha=0.6, label='Probabilidad Real')
    ax1.axvline(len_b1 + 0.5, color='grey', linestyle=':', lw=2, label='Punto de Cambio Real')
    ax1.set_ylabel('Probabilidad de Recompensa')
    ax1.set_title('Secuencia de Recompensas y Valor Predicho por el Modelo')
    ax1.legend(loc='upper right')
    ax1.grid(True, linestyle='--', alpha=0.5)

    # Gráfico 2: Tasa de Aprendizaje Dinámica
    ax2 = axes[1]
    ax2.plot(trials, simulator.history['learning_rates'], 'b-', label='Tasa de Aprendizaje ($\\alpha_t$)')
    ax2.axvline(len_b1 + 0.5, color='grey', linestyle=':', lw=2)
    ax2.set_ylabel('Tasa de Aprendizaje ($\\alpha$)')
    ax2.set_title('Adaptación de la Tasa de Aprendizaje')
    ax2.set_ylim(0, 1.1)
    ax2.grid(True, linestyle='--', alpha=0.5)

    # Gráfico 3: Probabilidad de Punto de Cambio (CPP)
    ax3 = axes[2]
    ax3.plot(trials, simulator.history['cpp'], 'm-', label='Probabilidad de Cambio ($CPP_t$)')
    ax3.axvline(len_b1 + 0.5, color='grey', linestyle=':', lw=2)
    ax3.set_xlabel('Número de Ensayo (Trial)')
    ax3.set_ylabel('Probabilidad')
    ax3.set_title('Creencia del Modelo sobre un Punto de Cambio')
    ax3.set_ylim(0, 1.1)
    ax3.grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# --- Creación de los Widgets Interactivos ---

style = {'description_width': 'initial'}
layout = Layout(width='500px')

# Sliders para el entorno
prob_b1_slider = FloatSlider(value=0.8, min=0.0, max=1.0, step=0.05, description='Prob. Recompensa Bloque 1:', style=style, layout=layout)
len_b1_slider = IntSlider(value=75, min=20, max=200, step=5, description='Duración Bloque 1:', style=style, layout=layout)
prob_b2_slider = FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05, description='Prob. Recompensa Bloque 2:', style=style, layout=layout)
len_b2_slider = IntSlider(value=75, min=20, max=200, step=5, description='Duración Bloque 2:', style=style, layout=layout)

# Slider para el parámetro del modelo
hazard_rate_slider = FloatSlider(value=0.01, min=0.001, max=0.2, step=0.005, description='Tasa de Peligro (Hazard Rate):', style=style, layout=layout, readout_format='.3f')

# Conectar los widgets a la función de simulación
interactive_simulation = interactive(run_and_plot_simulation,
                                     prob_b1=prob_b1_slider, len_b1=len_b1_slider,
                                     prob_b2=prob_b2_slider, len_b2=len_b2_slider,
                                     hazard_rate=hazard_rate_slider)

# Mostrar la interfaz de usuario
print("Ajusta los parámetros para ver cómo responde el modelo. La simulación se actualizará automáticamente.")
display(interactive_simulation)

Ajusta los parámetros para ver cómo responde el modelo. La simulación se actualizará automáticamente.


interactive(children=(FloatSlider(value=0.8, description='Prob. Recompensa Bloque 1:', layout=Layout(width='50…